# GOLA Google Colab Demo

[Open in Colab](https://colab.research.google.com/github/abhyudaymishr/GNOsLA/blob/main/notebooks/gola_colab_demo.ipynb)

This notebook installs `gola` from GitHub and runs dataset-backed visualizations in Colab.

In [ ]:
import sys
import subprocess

subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'git+https://github.com/abhyudaymishr/GNOsLA.git'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'plotly>=5.22'])
print('install complete')

In [ ]:
import sys
from pathlib import Path

import numpy as np
import plotly.io as pio
from IPython.display import IFrame, Image, display

from gola.visualization import DatasetAttentionConfig, save_dataset_sparse_geometric_attention
from gola.visualization_plotly import PlotlyKernelVisualizationConfig, save_plotly_kernel_figure

if 'google.colab' in sys.modules:
    pio.renderers.default = 'colab'

print('imports complete')

In [ ]:
demo = np.random.randn(2, 4, 64, 64, 3).astype('float32')
demo[:, :, 24:40, 24:40, :2] = 0.0

data_path = Path('/content/gola_demo.npy' if 'google.colab' in sys.modules else 'notebooks/_colab_demo.npy')
data_path.parent.mkdir(parents=True, exist_ok=True)
np.save(data_path, demo)
data_path

In [ ]:
png_out = Path('/content/gola_attention_demo.png' if 'google.colab' in sys.modules else 'notebooks/gola_attention_demo.png')
cfg = DatasetAttentionConfig(data_path=data_path, layout='STHWC', sample_idx=0, time_idx=0)
save_dataset_sparse_geometric_attention(png_out, config=cfg, dpi=180)
display(Image(filename=str(png_out)))
png_out

In [ ]:
html_out = Path('/content/gola_kernel_3d_demo.html' if 'google.colab' in sys.modules else 'notebooks/gola_kernel_3d_demo.html')
plotly_cfg = PlotlyKernelVisualizationConfig(
    mode='dataset',
    output_html=html_out,
    dataset=DatasetAttentionConfig(data_path=data_path, layout='STHWC', sample_idx=0, time_idx=0),
    query_x=0.3,
    query_y=0.5,
    random_query=False,
)
save_plotly_kernel_figure(plotly_cfg)
display(IFrame(src=str(html_out), width='100%', height=760))
html_out

## Optional: Hugging Face dataset metadata

The target PreGen file is large (~50 GB). Use dry-run metadata first.

In [ ]:
from huggingface_hub import hf_hub_download

info = hf_hub_download(
    repo_id='sage-lab/PreGen-NavierStokes-2D',
    filename='Geometry_Axis/FPO_Geometry_Easy_NoObstacle.npy',
    repo_type='dataset',
    dry_run=True,
)
info